# Masakhane - Traduction Automatique pour les Langues Africaines (via JoeyNMT)

## Note avant de debuter:
### - L'idée est que vous soyez capable de faire des changements legers à ce code pour avoir un resultat pour votre corpus de traduction. 

### - tl;dr: Les commentaires **"TODO"** vous donnent des directives sur ce que vous devez modifier

### - Si vous voulez avoir une idée de ce que vous faites, lisez le texte et jetez un coup d'œil aux liens.

### - Avec 100 époques, l'exécution dans Google Colab devrait prendre environ 7 heures.

### - Une fois que vous avez obtenu un résultat pour votre langue, veuillez joindre et envoyer par courriel votre notebook qui l'a généré à masakhanetranslation@gmail.com.

### - Si vous y tenez et que vous en avez l'occasion, il serait formidable de faire un bref historique de votre langue. Voir les exemples dans [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

### - Ce notebook est destiné à être utilisé avec des données parallèles personnalisées. Cela signifie que vous avez besoin de deux fichiers, l'un dans votre langue, l'autre en anglais, et les lignes dans les fichiers sont des traductions correspondantes.

## Pré-traitez vos données

Nous supposons ici que vous disposez déjà d'un jeu de données. Le format dans lequel nous allons le traiter ici exige que 
1. vous avez deux fichiers, un pour chaque langue
2. les fichiers sont alignés sur les phrases, ce qui signifie que chaque ligne doit correspondre à la même ligne dans l'autre fichier.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# TODO: Définissez vos langues source (source_language) et cible (target_language). Gardez à l'esprit que ces langues utilisent traditionnellement les codes de langue que l'on trouve ici :
# Ceux-ci deviendront également les suffixes de tous les fichiers de vocabulaire et de corpus utilisés tout au long du projet.
import os
source_language = "en"
target_language = "xh" 
lc = False  # Si Vrai, transformer les données en minuscules.
seed = 42  # Graine aléatoire pour le mélange des données.
tag = "baseline" # Donnez un nom unique à votre dossier - ceci afin de vous assurer que vous n'écrasez pas les modèles que vous avez déjà soumis.

os.environ["src"] = source_language # Les défini en bash également, puisque nous utilisons souvent des scripts bash
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# Ceci permet de l'enregistrer dans un dossier de notre gdrive !
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [ ]:
!echo ""$gdrive_path"

In [ ]:
# Installer opus-tools
! pip install opustools-pkg

In [ ]:
# TODO: specifier les chemins vers les fichiers ici
source_file = "my_file.en"
target_file = "my_file.xh"

# Ils doivent avoir la meme taille.
! wc -l "$source_file"
! wc -l "$target_file"

In [ ]:
# TODO: Pre-traitement! (OPTIONEL)

# Si vos données contiennent des symboles bizarres ou autres, vous voudrez faire un peu de nettoyage et de normalisation.
# Nous n'avons pas le code dans le notebook pour cela, mais vous pouvez utiliser sacremoses "normalize" par exemple pour normaliser la ponctuation : https://github.com/alvations/sacremoses.

# Nous appliquons la tokénisation pour séparer les signes de ponctuation des mots réels, séparer les mots aux traits d'union, etc.
# Si vos données sont déjà tokenisées, c'est génial ! Passez cette cellule.
# Sinon, nous pouvons utiliser sacremoses pour faire la tokenisation pour nous. 
# Nous avons besoin que les données soient tokenisées de façon à ce qu'elles correspondent à l'ensemble de test global.

! pip install sacremoses

tok_source_file = source_file+".tok"
tok_target_file = target_file+".tok"

# Tokeniser la source
! sacremoses -l "$source_language" tokenize < "$source_file" > "$tok_source_file"
# Tokeniser la cible
! sacremoses -l "$target_language" tokenize < "$target_file" > "$tok_target_file"

# Regardons l'effet de la tokenisation sur le texte.
! head "$source_file"*
! head "$target_file"*

# Modifiez les pointeurs vers nos fichiers de façon à ce que nous continuions à travailler avec les données tokenisées.
source_file = tok_source_file
target_file = tok_target_file

In [ ]:
# Telecharger le jeu de données de test global.
! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-any.en
  
# Et le jeu de données de test specifique pour cette paire de langues.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

# TODO: Si cela échoue, cela signifie qu'il n'y a pas encore de jeu de test pour votre langue. C'est à vous d'en créer un.
# Une bonne idée serait de prendre un sous-ensemble aléatoire de vos données, et de l'ajouter à https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-any.en.
# Faites une Pull Request et faites-la approuvée et fusionnée.
# Puis répétez cette cellule pour récupérer le nouvel ensemble de test.
# Puis passez à la cellule suivante qui filtrera tous les doublons de l'ensemble d'entraînement, de sorte qu'il n'y ait pas de chevauchement entre l'ensemble d'entraînement et l'ensemble de test.

In [ ]:
# Lire les données de test pour filtrer les données d'entrainement et de développement.
# Stocker la partie anglaise dans le jeu pour des vérifications rapides du filtrage.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print("Chargement de {} phrases de test globales à filtrer à partir des données d'entrainement/développement REUSSI!".format(j))

In [ ]:
import pandas as pd

source = []
target = []
skip_lines = []  # Collectez les numéros de ligne de la partie source pour sauter les mêmes lignes pour la partie cible.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Sauter les phrases qui sont contenues dans l'ensemble de test.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Ajouter au corpus uniquement si la source correspondante n'a pas été sautée.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Chargement de données réussi! \nSaut de {}/{} lignes car contenues dans le jeu de test.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
df.head(3)

## Pré-traitement et exportation

C'est généralement une bonne idée de supprimer les traductions en double et les traductions contradictoires du corpus. En pratique, ces corpus publics en comportent un certain nombre qui doivent être nettoyés.

De plus, nous allons diviser nos données dans dev/test/train et les exporter vers le système de fichiers.

In [ ]:
# supprimer les traductions en double
df_pp = df.drop_duplicates()

# abandonner les traductions contradictoires
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Mélangez les données pour éliminer les biais dans la sélection du jeu de données de dev.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [ ]:
# Installez fuzzy wuzzy pour supprimer les phrases "presque en double" dans les jeux de test et d'entraînement.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# réinitialiser l'index du jeu d'entrainement après le filtrage précédent
df_pp.reset_index(drop=False, inplace=True)

# Supprimez les échantillons du jeu de données d'entrainement s'ils "chevauchent presque" 
# les échantillons de l'ensemble de test.

# Fonction de filtrage. Ajustez le tampon (pad) pour réduire les correspondances 
# candidates dans un certain intervalle de longueur de caractères de l'échantillon donné.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

In [ ]:
start_time = time.time()
### itérer sur les lignes d'un dataframe pandas n'est pas recommandé, utilisons le traitement multiple pour appliquer la fonction.

with Pool(cpu_count()-1) as pool:
    scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
hours, rem = divmod(time.time() - start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# Filtrer les "échantillons qui se chevauchent presque".
df_pp = df_pp.assign(scores=scores)
df_pp = df_pp[df_pp['scores'] < 95]

In [ ]:
# Cette section effectue la séparation entre train/dev pour les corpus parallèles et les enregistre dans des fichiers séparés.
# Nous utilisons 1000 dev test et l'ensemble de test donné.
import csv

# TODO: si votre corpus est inférieur à 1000, réduisez ce nombre. Avec un corpus aussi petit, vous risquez de ne pas obtenir de bons résultats avec NMT :/
# Faites le split entre dev/train et créez des corpus parallèles
num_dev_patterns = 1000

# Optionnel : mettre les corpus en minuscules - cela facilitera la généralisation, mais sans la casse correcte.
if lc:  # Julia : rendre la minusculisation facultative
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: les jeux de test déjà générés
dev = df_pp.tail(num_dev_patterns) # Herman: Erreur dans l'original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)


# TODO: Vérifiez le format ci-dessous. Il ne doit pas y avoir de guillemets supplémentaires ou de caractères bizarres. Il ne doit pas non plus être vide.
! head train.*
! head dev.*



---


## Installation de JoeyNMT

JoeyNMT est un paquet de Traduction Automatique Neuronale (NMT) simple et minimaliste, utile pour l'apprentissage et l'enseignement. Consultez la documentation de JoeyNMT [ici](https://joeynmt.readthedocs.io)  

In [ ]:
# Installer JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Installer Pytorch avec support GPU v1.7.1.
! pip install torch==1.7.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Pré-traitement des données en tokens BPE

- L'une des améliorations les plus puissantes pour les langues agglutinantes (une caractéristique de la plupart des langues bantoues) est l'utilisation de la tokenisation BPE [(Sennrich, 2015)](https://arxiv.org/abs/1508.07909).

- Il a également été démontré qu'en optimisant le nombre de codes BPE, nous améliorons considérablement les résultats pour les langues à faibles ressources [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Ci-dessous nous avons les scripts pour faire la tokenisation BPE de nos données. Nous utilisons 4000 tokens comme recommandé par [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). Vous n'avez pas besoin de modifier quoi que ce soit. Il suffit d'exécuter ce qui suit. 

In [ ]:
# L'utilisation d'une méthode différente de tokénisation a permis d'améliorer considérablement les performances de NMT. 
# Habituellement, NMT procède à la tokenisation par mots. Cependant, l'utilisation d'une méthode appelée BPE a permis d'améliorer considérablement les performances.

# Faire un NMT de sous-mots
from os import path
os.environ["src"] = source_language # Les définir en bash également, puisque nous utilisons souvent des scripts bash
os.environ["tgt"] = target_language

# Apprendre les BPE sur les données d'entrainement.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Appliquer les splits BPE aux données de développement et de test.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Créer un répertoire, déplacer tout ce à quoi nous tenons vers le bon endroit.
! mkdir -p "$data_path"
! cp train.* "$data_path"
! cp test.* "$data_path"
! cp dev.* "$data_path"
! cp bpe.codes.4000 "$data_path"
! ls "$data_path"

# Déplacez également tout ce qui nous intéresse vers un emplacement monté dans google drive (pertinent si vous travaillez sur colab) à gdrive_path.
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Créer le vocabulaire avec build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Echantillon de resultat
! echo "BPE Test language Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

In [ ]:
# Déplacez également tout ce qui nous intéresse vers un emplacement monté dans google drive (pertinent si vous travaillez sur colab) à gdrive_path.
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Creation de la config de JoeyNMT

JoeyNMT nécessite une configuration yaml. Nous fournissons un modèle ci-dessous. Nous avons également défini un certain nombre de valeurs par défaut, avec lesquelles vous pouvez jouer !

- Nous avons utilisé l'architecture Transformer 
- Nous avons fixé notre dropout à un niveau raisonnablement élevé : 0,3 (recommandé dans [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Des choses avec lesquelles il vaut la peine de s'amuser :
- La taille du lot (batch size) (il est également recommandé de la modifier pour les langues à faibles ressources).
- Le nombre d'époques (epochs) (nous l'avons fixé à 30 pour qu'il fonctionne en une heure environ, à des fins de test).
- Les options du décodeur (beam_size, alpha)
- Les métriques d'évaluation (BLEU versus Crhf4)

In [ ]:
# Ceci crée le fichier de configuration pour notre système JoeyNMT. Comme cela peut sembler compliqué, nous avons fourni quelques paramètres utiles à mettre à jour.
# (Vous pouvez bien sûr jouer avec tous les paramètres si vous le souhaitez !)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Créer la config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0
    sacrebleu:                      # options sacrebleu
        remove_whitespace: True     # Option `remove_whitespace` dans la fonction sacrebleu.corpus_chrf() (valeur par defaut: True)
        tokenize: "none"            # Option `tokenize` dans la fonction sacrebleu.corpus_bleu() (les options incluent : "none" (à utiliser pour les données de test déjà tokenisées), "13a" (tokenizer minimal par défaut), "intl" qui fait surtout de la ponctuation et de l'unicode, etc). 

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: essayez de passer de plateau au scheduling de Noam
    patience: 5                     # Pour le plateau : diminuer le taux d'apprentissage (learning rate) par le facteur de diminution (decrease_factor) si le score de validation ne s'est pas amélioré pendant ce nombre de tours de validation.
    learning_rate_factor: 0.5       # facteur pour le scheduler Noam (utilisé avec Transformer)
    learning_rate_warmup: 1000      # étapes d'échauffement pour le scheduler Noam (utilisé avec Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Diminuer pour faire des essais et vérifier le fonctionnement. Une trentaine de minutes suffisent pour vérifier si le système fonctionne.
    validation_freq: 1000          # TODO: Définir à au moins une fois par époque.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Mettez la valeur True si vous voulez écraser les modèles éventuellement existants. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Augmentez à 8 pour les données plus volumineuses.
        embeddings:
            embedding_dim: 256   # TODO: Augmentez à 512 pour les données plus volumineuses.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Augmentez à 512 pour les données plus volumineuses.
        ff_size: 1024            # TODO: Augmentez à 2048 pour les données plus volumineuses.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Augmentez à 8 pour les données plus volumineuses.
        embeddings:
            embedding_dim: 256    # TODO: Augmentez à 512 pour les données plus volumineuses.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Augmentez à 512 pour les données plus volumineuses.
        ff_size: 1024            # TODO: Augmentez à 2048 pour les données plus volumineuses.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Entrainer le Modele

Cette simple ligne de joeynmt exécute l'entraînement en utilisant la configuration que nous avons faite ci-dessus

In [ ]:
# Entrainer le modèle
# Vous pouvez appuyer sur Ctrl-C pour arrêter. Et ensuite, exécutez la cellule suivante pour sauvegarder vos points de contrôle !
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

In [ ]:
# Copiez les modèles créés depuis le stockage du notebook vers google drive pour un stockage permanent. 
! mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"
! cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [ ]:
# Sortie de notre score de validation
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

In [ ]:
# Tester notre modèle
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"